In [13]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from ragas.metrics import (
    answer_relevancy,
    context_precision,
    context_recall,
    faithfulness
)
from ragas import evaluate
from ragas import SingleTurnSample

from datasets import Dataset
import json
import os

In [14]:
file = open('data/key.txt', 'r')
# Read the API key from the file
api_key = file.read().strip()
# Close the file
file.close()
# 🧠 Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = api_key

# 2️⃣ Load the Ramayana PDF
loader = PyMuPDFLoader("data/RAMAYANA.pdf")
docs = loader.load()

# 3️⃣ Embed and store in vector DB
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2",
                                        model_kwargs={"device": "cpu"})
vectorstore = Chroma.from_documents(docs, embedding=embedding)

# 4️⃣ Set up QA Chain
retriever = vectorstore.as_retriever()
llm = ChatOpenAI(model_name="gpt-4", temperature=0.2)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)


In [15]:
# 4️⃣ Load Evaluation Questions
queries = [
  {
    "question": "Who was the father of Lord Rama?",
    "ground_truth": "King Dasharatha was the father of Lord Rama."
  },
  {
    "question": "Why did Rama go to the forest?",
    "ground_truth": "Rama went to the forest to honor his father's promise to Kaikeyi."
  }
]



In [ ]:
# 6️⃣ Run RAG and create Samples
samples = []

for item in queries:
    query = item["question"]
    ground_truth = item["ground_truth"]

    
    result = qa_chain(query)
    answer = result["result"]
    contexts = [doc.page_content for doc in result["source_documents"]]

    print(f"🔍 Query: {query}")
    print(f"💬 Answer: {answer}")
    print(f"📄 Contexts: {contexts}\n")
    print(f"✅ Ground Truth: {ground_truth}\n")

    # Evaluation rubric
    rubric = {
        "accuracy": "Correct",
        "completeness": "High",
        "fluency": "Excellent"
    }

    # Create the SingleTurnSample instance
    sample = SingleTurnSample(
        user_input=query,
        retrieved_contexts=contexts,
        response=answer,
        reference=ground_truth,
        rubric=rubric
    )
    samples.append(sample)
    print(sample.to_dict())


🔍 Query: Who was the father of Lord Rama?
💬 Answer: The father of Lord Rama was King Dasharatha.
📄 Contexts: ['16 \n \nthe time he took him to the blind parents, the boy was already dead. \nThe grief-stricken father of Sravanakumar cursed Dasahratha “O \nKing! You have killed my son and thus separated me from him. This \ngrief will be the cause of my death. You shall also die, pining for \nyour son. \nDasahratha told Kaushalya about this incident and holding her \nhands, said “The sage’s curse had come true now. Please forgive me \nfor all my sins”.  \nThe king recalled each and every feature of the handsome Rama and \nwith tears blurring his eyes cried out “I wanted my son to be near \nme when I die. Kaikeyi has destroyed even this. She is my true \nenemy. Kaushalya! Sumitra! I do not want to live longer. My eyes are \nfalling. Rama, my son! Rama, my son!” The king died. \nAyodhya which was still grieving over Rama’s exile was plunged into \nsorrow as the news of the king’s death spre

In [20]:
# 7️⃣ Convert to RAGAS-compatible HuggingFace Dataset
ragas_dataset = Dataset.from_list([sample.to_dict() for sample in samples])
print("📦 RAGAS Dataset Created with {} samples.".format(len(ragas_dataset)))

# 8️⃣ Evaluate with RAGAS
result = evaluate(
    ragas_dataset,
    metrics=[
        answer_relevancy,
        context_precision,
        context_recall,
        faithfulness
    ]
)

# 9️⃣ Display Results
print("🔍 Evaluation Results:\n")
print(result.to_pandas())

📦 RAGAS Dataset Created with 2 samples.


Evaluating: 100%|██████████| 8/8 [00:13<00:00,  1.69s/it]


🔍 Evaluation Results:

                         user_input  \
0  Who was the father of Lord Rama?   
1    Why did Rama go to the forest?   

                                  retrieved_contexts  \
0  [16 \n \nthe time he took him to the blind par...   
1  [12 \n \nRama to the forest? Is she afraid of ...   

                                            response  \
0       The father of Lord Rama was King Dasharatha.   
1  Rama went to the forest to carry out his fathe...   

                                           reference  answer_relevancy  \
0       King Dasharatha was the father of Lord Rama.               1.0   
1  Rama went to the forest to honor his father's ...               1.0   

   context_precision  context_recall  faithfulness  
0                1.0             1.0      1.000000  
1                1.0             1.0      0.666667  
